In [1]:
import torch
import torch.nn as nn

from utils import load_and_preprocess_data

from EncoderBlock import Encoder

from PositionalEncoding import PositionalEncoding
from MultiHeadAttention import MHA
from AddNorm import AddNorm
from PositionwiseFeedForward import PositionwiseFeedForward

# Load and Preprocess data

In [2]:
data, vocab  = load_and_preprocess_data()

tokenize = {char : index for index, char in enumerate(vocab) }
detokenize = {char : index for char, index in enumerate (list(tokenize.keys()))}

tokens = []

for char in data:
    tokens.append(tokenize[char])

embedding_dim = 50

vocab_size = len(tokenize.keys())

In [3]:
e = Encoder(vocab_size=vocab_size, embedding_dim=embedding_dim)

In [4]:
x = torch.randint(1, 10, (2, 12))

In [5]:
o = e(x)
o.shape

Shape after embedding: torch.Size([2, 12, 50])
Shape after positional encoding: torch.Size([1, 12, 50])
Shape after multi-head attention: torch.Size([1, 12, 50])
Shape after add-norm: torch.Size([1, 12, 50])
Shape after position-wise feed forward: torch.Size([1, 12, 50])
Shape after second add-norm: torch.Size([1, 12, 50])


torch.Size([1, 12, 50])